# Crimson Powder Analysis

This is some code to determine some parameters needed for the ejection charge. We're using `pypropep` to analyze as it's really powerful and cool.

We'll initialize our project by importing some packages.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import operator
import pprint
from matplotlib import pyplot as plt
from IPython.display import Math

Since we run into some incompatibility issues with our import since the `collections.abc` package hadn't been patched this is some code to fix that.

In [2]:
# Solving the collections module compatibility issue
import collections.abc
import collections
if not hasattr(collections, 'MutableMapping'):
    collections.MutableMapping = collections.abc.MutableMapping
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping
if not hasattr(collections, 'Sequence'):
    collections.Sequence = collections.abc.Sequence

Now we can finally import `pypropep`.

In [3]:
import pypropep as ppp
import shutil
import os

We'll initialize `matplotlib` by running the following:

In [4]:
plt.style.use(u'ggplot')
plt.rcParams['figure.figsize'] = (10,6)
ppp.init()

Loaded 1921 thermo species
Loaded 1032 propellants


This is unnecessary but here we'll show how powerful this package is.

In [5]:
print(ppp.PROPELLANT_FILE)
print(ppp.THERMO_FILE)

/home/azid0/.pyenv/versions/.nakuja/lib/python3.11/site-packages/pypropep/data/propellant.dat
/home/azid0/.pyenv/versions/.nakuja/lib/python3.11/site-packages/pypropep/data/thermo.dat


We see that our crimson powder made from potassium nitrate($\mathbf{KNO_3}$), ascorbic acid a.k.a *vitamin C*($\mathbf{C_{6}H_{8}O_{6}}$) and ferric oxide ($\mathbf{Fe_{2}O_{3}}$).

$\mathbf{KNO_3}$ and $\mathbf{Fe_{2}O_{3}}$ are in the `propellant.dat` file, but $\mathbf{C_{6}H_{8}O_{6}}$ isn't. So we'll add our entry here

Basically, the line reads:
- Line number: 1100
- Name: ASCORBIC ACID
- Formula: C6H8O6 formatted as the pairs:
    - 6C (carbon)
    - 8H (hydrogen)
    - 6O (oxygen)
    - 0 (empty)
    - 0 (empty)
    - 0 (empty)
- Heat of formation: -302 cal/g
- Density: 0.0596 lb/in³

In [6]:
pprint.pprint(ppp.find_propellant('ascorbic'))

[Propellant: C6H8O6 - ASCORBIC ACID [1031]]


Now we'll perform some equilibrium calculations to find the **Adiabatic Flame Temperature** of our charge mix:

$$
4\mathbf{KNO_3}+\mathbf{C_{6}H_{8}O_{6}}->2\mathbf{K_{2}CO_3}+2\mathbf{CO_2}+4\mathbf{N_2}+4\mathbf{H_{2}O}
$$

We'll firs

In [7]:
kno3 = ppp.PROPELLANTS['POTASSIUM NITRATE']
c6h8o6 = ppp.PROPELLANTS['ASCORBIC ACID']
fe2o3 = ppp.PROPELLANTS['FERRIC OXIDE HEMATITE']

e1 = ppp.Equilibrium()
e1.add_propellants([(kno3, 0.06), (c6h8o6, 0.02), (fe2o3, 0.003)])
e1.set_state(T=273, P=2., )

for station in ['chamber', 'exit']:
    print(f"{station} : ")
    pprint.pprint(p.composition[station][:-1], indent=4)
    print("Condensed: ")
    pprint.pprint(p.composition_condensed[station], indent=4)
    print('\n')

ValueError: set_state: Temperature must not be specified                     for mode 'HP' or 'SP'

In [ ]:
e3 = ppp.Equilibrium()
e3.add_propellants([(kno3, 0.06), (c6h8o6, 0.02), (fe2o3, 0.003)])
e3.set_state(T=273,P=1., type="TP")

print(f"{e3}")

Status:
	Equillibrium Computed: True
	Properties Computed: True
	Performance Computed: False
Composition:
	POTASSIUM NITRATE - 0.060 mol
	ASCORBIC ACID - 0.020 mol
	FERRIC OXIDE HEMATITE - 0.003 mol
State:
	Pressure: 1.000 atm 
	Temperature: 273.0 K 
	Enthalpy: -8496.504 kJ/kg 
	Int. Energy: -8519.072 kJ/kg 
	Gibbs Free Energy: -9278.642 kJ/kg 
	Entropy: 2.865 kJ/kg-K 
	Molar Mass: 100.580 g/mol 
	dV_P: -1.006
	dV_T: 1.135
	Cp: 1.248 kJ/kg-K
	Cv: 1.142 kJ/kg-K
	gamma: 1.086
	Sound Speed: 156.6 m/s
	
	


In [9]:
kno3 = ppp.PROPELLANTS['POTASSIUM NITRATE']
c6h8o6 = ppp.PROPELLANTS['ASCORBIC ACID']
species = ['CO2', 'CO', 'H2O', 'N2', 'H2', 'H', 'OH', 'K2CO3']
species_log = ['H2','CO2', 'N2', 'K2CO3', 'CO']
phi = np.linspace(0.5, 1.5, 50000)
T_adiabatic = []
N_O_atoms = kno3.atoms_of('O') + c6h8o6.atoms_of('O')
N_C_atoms = c6h8o6.atoms_of('C')
N_H_atoms = c6h8o6.atoms_of('H')
N_K_atoms = kno3.atoms_of('K')
N_ratio_st = N_O_atoms / (N_C_atoms / 2 + N_H_atoms * 2 + N_K_atoms * 2)

comp = dict()
for s in (species + species_log):
    comp[s] = []
for i in range(len(phi)):
    e3 = ppp.Equilibrium()
    n_fuel = phi[i] * N_ratio_st
    e3.add_propellants([(c6h8o6, n_fuel), (kno3, 4.)])
    e3.set_state(P=1., type='HP')
    T_adiabatic.append(e3.properties.T)
    for s in (species + species_log):
        comp[s].append(e3.composition[s])

plt.figure(figsize=(10,12))
ax = plt.subplot(311)
plt.plot(phi, T_adiabatic)
plt.ylabel("Temperature (K)")

plt.subplot(312,sharex=ax)
for s in species:
    plt.plot(phi, comp[s], label=s)
plt.ylabel("Composition")
plt.legend(loc='best', fontsize='small')

plt.subplot(313,sharex=ax)
for s in (species + species_log):
    plt.semilogy(phi, comp[s], label=s)
plt.xlabel(r"$\phi$")
plt.ylabel("Composition")
plt.legend(loc='best', fontsize='small')

RuntimeError: Equilibrium failed with error: 'Too many equilibrium iterations'